In [1]:
# Cell 1: Setup and Configuration
import pandas as pd
import numpy as np
import json
import pickle
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Data directories
SEC_DIR = os.path.join('..', 'data', 'raw', 'kaggle', 'sec_edgar')
FACTS_DIR = os.path.join(SEC_DIR, 'companyfacts')
PRICE_DIR = os.path.join('..', 'data', 'raw', 'kaggle', 'individual_stocks_5yr')
DATA_DIR = os.path.join('..', 'data')

# 50 S&P 500 companies - large cap, liquid, diverse sectors
# Using tickers that match the Kaggle price data filenames (2013-2018 era)
TARGET_TICKERS = [
    # Tech
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'INTC', 'NVDA', 'CSCO', 'ORCL', 'ADBE',
    # Finance
    'JPM', 'BAC', 'WFC', 'GS', 'MS', 'V', 'MA', 'AXP', 'BLK', 'SPGI',
    # Healthcare
    'UNH', 'JNJ', 'PFE', 'ABBV', 'MRK', 'ABT', 'TMO', 'MDT', 'AMGN', 'GILD',
    # Consumer
    'PG', 'KO', 'PEP', 'WMT', 'COST', 'HD', 'NKE', 'MCD', 'DIS', 'NFLX',
    # Industrial / Energy / Other
    'XOM', 'CVX', 'BA', 'CAT', 'HON', 'UNP', 'RTX', 'GE', 'MMM', 'IBM'
]

print(f'Target: {len(TARGET_TICKERS)} companies')
print(f'SEC data dir: {FACTS_DIR}')
print(f'Price data dir: {PRICE_DIR}')

# Verify directories exist
print(f'\nSEC files found: {len(os.listdir(FACTS_DIR))}')
print(f'Price files found: {len(os.listdir(PRICE_DIR))}')

Target: 50 companies
SEC data dir: ..\data\raw\kaggle\sec_edgar\companyfacts
Price data dir: ..\data\raw\kaggle\individual_stocks_5yr

SEC files found: 19215
Price files found: 505


In [2]:
# Cell 2: Load Stock Prices from Kaggle
def load_prices(tickers, price_dir):
    """Load price data from Kaggle CSVs."""
    price_data = {}
    missing = []

    for ticker in tickers:
        filepath = os.path.join(price_dir, f'{ticker}_data.csv')
        if not os.path.exists(filepath):
            missing.append(ticker)
            continue

        df = pd.read_csv(filepath)
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')

        # Standardize column names
        col_map = {}
        for col in df.columns:
            cl = col.strip().lower()
            if cl == 'open': col_map[col] = 'Open'
            elif cl == 'high': col_map[col] = 'High'
            elif cl == 'low': col_map[col] = 'Low'
            elif cl == 'close': col_map[col] = 'Close'
            elif cl == 'volume': col_map[col] = 'Volume'
            elif cl == 'adj close': col_map[col] = 'Adj Close'
        df = df.rename(columns=col_map)

        # Ensure Adj Close exists
        if 'Adj Close' not in df.columns:
            df['Adj Close'] = df['Close']

        # Drop non-standard columns (like 'Name')
        keep_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
        df = df[[c for c in keep_cols if c in df.columns]]

        price_data[ticker] = df

    print(f'Loaded prices for {len(price_data)} companies')
    if missing:
        print(f'Missing price data: {missing}')

    # Show date range
    sample = list(price_data.values())[0]
    print(f'Date range: {sample.index.min().date()} to {sample.index.max().date()}')

    return price_data

price_data = load_prices(TARGET_TICKERS, PRICE_DIR)

# Show which tickers we have
available_tickers = list(price_data.keys())
print(f'\nAvailable tickers ({len(available_tickers)}): {available_tickers}')

Loaded prices for 49 companies
Missing price data: ['RTX']
Date range: 2013-02-08 to 2018-02-07

Available tickers (49): ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'INTC', 'NVDA', 'CSCO', 'ORCL', 'ADBE', 'JPM', 'BAC', 'WFC', 'GS', 'MS', 'V', 'MA', 'AXP', 'BLK', 'SPGI', 'UNH', 'JNJ', 'PFE', 'ABBV', 'MRK', 'ABT', 'TMO', 'MDT', 'AMGN', 'GILD', 'PG', 'KO', 'PEP', 'WMT', 'COST', 'HD', 'NKE', 'MCD', 'DIS', 'NFLX', 'XOM', 'CVX', 'BA', 'CAT', 'HON', 'UNP', 'GE', 'MMM', 'IBM']


In [3]:
# Cell 3: Load Ticker-to-CIK Mapping
def load_ticker_to_cik(filepath):
    """Load SEC ticker to CIK number mapping."""
    with open(filepath) as f:
        data = json.load(f)

    ticker_to_cik = {}
    for item in data.values():
        ticker = item['ticker']
        cik = str(item['cik_str']).zfill(10)
        ticker_to_cik[ticker] = cik

    return ticker_to_cik

ticker_to_cik = load_ticker_to_cik(os.path.join(SEC_DIR, 'company_tickers.json'))

print(f'Total tickers in SEC mapping: {len(ticker_to_cik)}')

# Check which of our target tickers have SEC data
found = []
not_found = []
for t in available_tickers:
    if t in ticker_to_cik:
        cik = ticker_to_cik[t]
        json_path = os.path.join(FACTS_DIR, f'CIK{cik}.json')
        if os.path.exists(json_path):
            found.append(t)
        else:
            not_found.append(f'{t} (CIK file missing)')
    else:
        not_found.append(f'{t} (no CIK mapping)')

print(f'\nTickers with price + SEC data: {len(found)}')
if not_found:
    print(f'Missing SEC data: {not_found}')

Total tickers in SEC mapping: 10379

Tickers with price + SEC data: 48
Missing SEC data: ['FB (no CIK mapping)']


In [4]:
# Cell 4: SEC EDGAR Parser

# XBRL tag mapping: our metric name -> list of possible SEC tags (tried in order)
INCOME_TAGS = {
    'Total Revenue': [
        'RevenueFromContractWithCustomerExcludingAssessedTax',
        'Revenues',
        'SalesRevenueNet',
        'SalesRevenueGoodsNet',
        'RevenueFromContractWithCustomerIncludingAssessedTax',
    ],
    'Cost of Revenue': [
        'CostOfGoodsAndServicesSold',
        'CostOfRevenue',
        'CostOfGoodsSold',
    ],
    'Gross Profit': ['GrossProfit'],
    'Operating Income': ['OperatingIncomeLoss'],
    'Net Income': ['NetIncomeLoss'],
    'EPS Basic': ['EarningsPerShareBasic'],
    'EPS Diluted': ['EarningsPerShareDiluted'],
    'R&D Expense': ['ResearchAndDevelopmentExpense'],
    'SGA Expense': [
        'SellingGeneralAndAdministrativeExpense',
        'SellingAndMarketingExpense',
    ],
    'Interest Expense': [
        'InterestExpense',
        'InterestExpenseDebt',
    ],
    'Income Tax': ['IncomeTaxExpenseBenefit'],
}

BALANCE_TAGS = {
    'Total Assets': ['Assets'],
    'Current Assets': ['AssetsCurrent'],
    'Total Liabilities': ['Liabilities', 'LiabilitiesAndStockholdersEquity'],
    'Current Liabilities': ['LiabilitiesCurrent'],
    'Long Term Debt': ['LongTermDebtNoncurrent', 'LongTermDebt'],
    'Short Term Debt': ['ShortTermBorrowings', 'CommercialPaper'],
    'Stockholders Equity': ['StockholdersEquity'],
    'Cash': ['CashAndCashEquivalentsAtCarryingValue', 'CashCashEquivalentsAndShortTermInvestments'],
    'Shares Outstanding': ['CommonStockSharesOutstanding'],
}

CASHFLOW_TAGS = {
    'Operating Cash Flow': [
        'NetCashProvidedByUsedInOperatingActivities',
        'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations',
    ],
    'Capital Expenditures': [
        'PaymentsToAcquirePropertyPlantAndEquipment',
        'PaymentsToAcquireProductiveAssets',
    ],
}

def extract_quarterly_values(facts, tag_names, is_instant=False):
    """
    Extract single-quarter values from SEC XBRL data.
    
    For flow metrics (income, cash flow): filter for ~90-day periods.
    For instant metrics (balance sheet): use end date directly.
    """
    gaap = facts.get('us-gaap', {})

    for tag in tag_names:
        if tag not in gaap:
            continue

        # Determine unit type
        units = gaap[tag].get('units', {})
        if 'USD' in units:
            entries = units['USD']
        elif 'shares' in units:
            entries = units['shares']
        elif 'USD/shares' in units:
            entries = units['USD/shares']
        else:
            continue

        # Filter for 10-Q filings
        quarterly = [e for e in entries if e.get('form') == '10-Q']
        if not quarterly:
            continue

        values = {}
        for entry in quarterly:
            end_date = pd.Timestamp(entry['end'])

            if is_instant:
                # Balance sheet: point-in-time, no start date filtering needed
                values[end_date] = entry['val']
            else:
                # Income/cash flow: need single-quarter (not cumulative)
                start = entry.get('start')
                if start:
                    days = (datetime.strptime(entry['end'], '%Y-%m-%d') -
                            datetime.strptime(start, '%Y-%m-%d')).days
                    # Single quarter: 85-100 days
                    if 85 <= days <= 100:
                        values[end_date] = entry['val']

        if values:
            return values, tag

    return {}, None

print('SEC EDGAR parser defined')
print(f'Income tags: {len(INCOME_TAGS)} metrics')
print(f'Balance tags: {len(BALANCE_TAGS)} metrics')
print(f'Cash flow tags: {len(CASHFLOW_TAGS)} metrics')
print(f'Total: {len(INCOME_TAGS) + len(BALANCE_TAGS) + len(CASHFLOW_TAGS)} raw metrics')

SEC EDGAR parser defined
Income tags: 11 metrics
Balance tags: 9 metrics
Cash flow tags: 2 metrics
Total: 22 raw metrics


In [5]:
# Cell 5: Extract Financial Data for All Companies

def load_company_financials(ticker, cik, facts_dir):
    """Load and parse SEC financial data for one company."""
    json_path = os.path.join(facts_dir, f'CIK{cik}.json')
    with open(json_path) as f:
        data = json.load(f)

    facts = data['facts']

    # Extract income statement metrics (flow: need single-quarter filter)
    income_data = {}
    income_tags_used = {}
    for metric_name, tag_list in INCOME_TAGS.items():
        values, tag_used = extract_quarterly_values(facts, tag_list, is_instant=False)
        if values:
            income_data[metric_name] = values
            income_tags_used[metric_name] = tag_used

    # Extract balance sheet metrics (instant: point-in-time)
    balance_data = {}
    balance_tags_used = {}
    for metric_name, tag_list in BALANCE_TAGS.items():
        values, tag_used = extract_quarterly_values(facts, tag_list, is_instant=True)
        if values:
            balance_data[metric_name] = values
            balance_tags_used[metric_name] = tag_used

    # Extract cash flow metrics (flow: need single-quarter filter)
    cashflow_data = {}
    for metric_name, tag_list in CASHFLOW_TAGS.items():
        values, tag_used = extract_quarterly_values(facts, tag_list, is_instant=False)
        if values:
            cashflow_data[metric_name] = values

    # Convert to DataFrames (metrics as rows, dates as columns)
    # Income statement
    if income_data:
        income_df = pd.DataFrame(income_data)
        income_df.index = pd.to_datetime(income_df.index)
        income_df = income_df.sort_index()
        income_df = income_df.T  # Metrics as rows, dates as columns
    else:
        income_df = pd.DataFrame()

    # Balance sheet + cash flow combined
    all_balance = {**balance_data, **cashflow_data}
    if all_balance:
        balance_df = pd.DataFrame(all_balance)
        balance_df.index = pd.to_datetime(balance_df.index)
        balance_df = balance_df.sort_index()
        balance_df = balance_df.T  # Metrics as rows, dates as columns
    else:
        balance_df = pd.DataFrame()

    return {
        'ticker': ticker,
        'quarterly_income': income_df,
        'quarterly_balance': balance_df,
        'info': {'symbol': ticker},
        '_income_tags': income_tags_used,
        '_balance_tags': balance_tags_used,
    }

# Load all companies
print('Loading SEC financial data...')
financial_data = []

for ticker in found:
    cik = ticker_to_cik[ticker]
    try:
        company = load_company_financials(ticker, cik, FACTS_DIR)

        income_metrics = len(company['quarterly_income'])
        balance_metrics = len(company['quarterly_balance'])
        quarters = company['quarterly_income'].shape[1] if not company['quarterly_income'].empty else 0

        # Only include companies with minimum data
        if income_metrics >= 3 and quarters >= 5:
            financial_data.append(company)
            print(f'  {ticker}: {income_metrics} income + {balance_metrics} balance metrics, {quarters} quarters')
        else:
            print(f'  {ticker}: SKIPPED (only {income_metrics} income metrics, {quarters} quarters)')

    except Exception as e:
        print(f'  {ticker}: ERROR - {e}')

print(f'\nLoaded {len(financial_data)} companies with sufficient data')

Loading SEC financial data...
  AAPL: 11 income + 11 balance metrics, 53 quarters
  MSFT: 11 income + 11 balance metrics, 53 quarters
  GOOGL: 10 income + 11 balance metrics, 34 quarters
  AMZN: 9 income + 11 balance metrics, 53 quarters
  INTC: 11 income + 11 balance metrics, 53 quarters
  NVDA: 11 income + 10 balance metrics, 53 quarters
  CSCO: 11 income + 11 balance metrics, 52 quarters
  ORCL: 11 income + 9 balance metrics, 53 quarters
  ADBE: 10 income + 10 balance metrics, 53 quarters
  JPM: 6 income + 7 balance metrics, 53 quarters
  BAC: 6 income + 8 balance metrics, 53 quarters
  WFC: 6 income + 7 balance metrics, 53 quarters
  GS: 5 income + 9 balance metrics, 53 quarters
  MS: 6 income + 8 balance metrics, 53 quarters
  V: 5 income + 10 balance metrics, 53 quarters
  MA: 7 income + 9 balance metrics, 53 quarters
  AXP: 7 income + 9 balance metrics, 52 quarters
  BLK: 6 income + 8 balance metrics, 7 quarters
  SPGI: 9 income + 11 balance metrics, 53 quarters
  UNH: 9 income 

In [6]:
# Cell 6: Data Quality Report

print('=' * 60)
print('DATA QUALITY REPORT')
print('=' * 60)

# Price data summary
print(f'\n--- Price Data ---')
print(f'Companies: {len(price_data)}')
sample_prices = list(price_data.values())[0]
print(f'Date range: {sample_prices.index.min().date()} to {sample_prices.index.max().date()}')

# Financial data summary
print(f'\n--- Financial Data ---')
print(f'Companies: {len(financial_data)}')

# Metric coverage
all_income_metrics = set()
all_balance_metrics = set()
for company in financial_data:
    all_income_metrics.update(company['quarterly_income'].index.tolist())
    all_balance_metrics.update(company['quarterly_balance'].index.tolist())

print(f'\nIncome metrics found: {sorted(all_income_metrics)}')
print(f'Balance/CF metrics found: {sorted(all_balance_metrics)}')

# Per-metric coverage across companies
print(f'\n--- Metric Coverage (% of companies) ---')
for metric in sorted(all_income_metrics):
    count = sum(1 for c in financial_data if metric in c['quarterly_income'].index)
    pct = count / len(financial_data) * 100
    print(f'  {metric:<30s} {count:>3}/{len(financial_data)} ({pct:.0f}%)')

for metric in sorted(all_balance_metrics):
    count = sum(1 for c in financial_data if metric in c['quarterly_balance'].index)
    pct = count / len(financial_data) * 100
    print(f'  {metric:<30s} {count:>3}/{len(financial_data)} ({pct:.0f}%)')

# Date overlap analysis
print(f'\n--- Date Overlap ---')
price_min = sample_prices.index.min()
price_max = sample_prices.index.max()
total_valid_quarters = 0

for company in financial_data:
    ticker = company['ticker']
    if company['quarterly_income'].empty:
        continue
    quarters = company['quarterly_income'].columns
    valid = sum(1 for q in quarters
                if price_min <= q <= price_max
                and price_min <= q + pd.Timedelta(days=90) <= price_max)
    total_valid_quarters += valid

print(f'Total valid company-quarters (with price overlap): {total_valid_quarters}')
print(f'Expected samples after feature engineering: ~{int(total_valid_quarters * 0.85)}')

DATA QUALITY REPORT

--- Price Data ---
Companies: 49
Date range: 2013-02-08 to 2018-02-07

--- Financial Data ---
Companies: 46

Income metrics found: ['Cost of Revenue', 'EPS Basic', 'EPS Diluted', 'Gross Profit', 'Income Tax', 'Interest Expense', 'Net Income', 'Operating Income', 'R&D Expense', 'SGA Expense', 'Total Revenue']
Balance/CF metrics found: ['Capital Expenditures', 'Cash', 'Current Assets', 'Current Liabilities', 'Long Term Debt', 'Operating Cash Flow', 'Shares Outstanding', 'Short Term Debt', 'Stockholders Equity', 'Total Assets', 'Total Liabilities']

--- Metric Coverage (% of companies) ---
  Cost of Revenue                 34/46 (74%)
  EPS Basic                       45/46 (98%)
  EPS Diluted                     45/46 (98%)
  Gross Profit                    15/46 (33%)
  Income Tax                      46/46 (100%)
  Interest Expense                40/46 (87%)
  Net Income                      46/46 (100%)
  Operating Income                35/46 (76%)
  R&D Expense  

In [7]:
# Cell 7: Save Data

# Save price data
with open(os.path.join(DATA_DIR, 'price_data.pkl'), 'wb') as f:
    pickle.dump(price_data, f)

# Save financial data (remove internal debug fields)
clean_financial = []
for company in financial_data:
    clean_financial.append({
        'ticker': company['ticker'],
        'quarterly_income': company['quarterly_income'],
        'quarterly_balance': company['quarterly_balance'],
        'info': company['info'],
    })

with open(os.path.join(DATA_DIR, 'financial_data.pkl'), 'wb') as f:
    pickle.dump(clean_financial, f)

# Save ticker list
final_tickers = [c['ticker'] for c in financial_data]
with open(os.path.join(DATA_DIR, 'tickers.pkl'), 'wb') as f:
    pickle.dump(final_tickers, f)

print('Data saved:')
print(f'  price_data.pkl - {len(price_data)} companies')
print(f'  financial_data.pkl - {len(clean_financial)} companies')
print(f'  tickers.pkl - {len(final_tickers)} tickers')
print(f'\nTickers: {final_tickers}')

Data saved:
  price_data.pkl - 49 companies
  financial_data.pkl - 46 companies
  tickers.pkl - 46 tickers

Tickers: ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'INTC', 'NVDA', 'CSCO', 'ORCL', 'ADBE', 'JPM', 'BAC', 'WFC', 'GS', 'MS', 'V', 'MA', 'AXP', 'BLK', 'SPGI', 'UNH', 'JNJ', 'PFE', 'ABBV', 'MRK', 'ABT', 'TMO', 'MDT', 'AMGN', 'GILD', 'PG', 'KO', 'WMT', 'HD', 'NKE', 'MCD', 'DIS', 'NFLX', 'XOM', 'CVX', 'BA', 'CAT', 'HON', 'UNP', 'GE', 'MMM', 'IBM']


In [8]:
# Cell 8: Verify Saved Data

# Reload and verify
with open(os.path.join(DATA_DIR, 'price_data.pkl'), 'rb') as f:
    test_prices = pickle.load(f)
with open(os.path.join(DATA_DIR, 'financial_data.pkl'), 'rb') as f:
    test_financials = pickle.load(f)

# Check first company
sample = test_financials[0]
ticker = sample['ticker']

print(f'=== Verification: {ticker} ===')
print(f'\nPrice data:')
print(test_prices[ticker].head())

print(f'\nQuarterly Income ({sample["quarterly_income"].shape}):')
print(sample['quarterly_income'].iloc[:, :5])

print(f'\nQuarterly Balance ({sample["quarterly_balance"].shape}):')
print(sample['quarterly_balance'].iloc[:, :5])

print(f'\nAll metrics available:')
print(f'  Income: {list(sample["quarterly_income"].index)}')
print(f'  Balance/CF: {list(sample["quarterly_balance"].index)}')

print(f'\nData is ready for notebook 02 (feature engineering)!')

=== Verification: AAPL ===

Price data:
               Open     High      Low    Close     Volume  Adj Close
date                                                                
2013-02-08  67.7142  68.4014  66.8928  67.8542  158168416    67.8542
2013-02-11  68.0714  69.2771  67.6071  68.5614  129029425    68.5614
2013-02-12  68.5014  68.9114  66.8205  66.8428  151829363    66.8428
2013-02-13  66.7442  67.6628  66.1742  66.7156  118721995    66.7156
2013-02-14  66.3599  67.3771  66.2885  66.6556   88809154    66.6556

Quarterly Income ((11, 53)):
                    2008-06-28    2008-12-27    2009-03-28    2009-06-27  \
Total Revenue              NaN           NaN           NaN           NaN   
Cost of Revenue   4.864000e+09  7.373000e+09  5.457000e+09  5.751000e+09   
Gross Profit      2.600000e+09  4.507000e+09  3.627000e+09  3.983000e+09   
Operating Income  1.392000e+09  3.101000e+09  2.323000e+09  2.632000e+09   
Net Income        1.072000e+09  2.255000e+09  1.620000e+09  1.82800